In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install h2o

     |████████████████████████████████| 164.6MB 95kB/s 
  Created wheel for h2o: filename=h2o-3.32.0.4-py2.py3-none-any.whl size=164670979 sha256=34d06d06271f6c9835b1b50cf374cf972872820d503c9574a26f2923483aef2b
  Stored in directory: /root/.cache/pip/wheels/65/f4/0d/c9bb958d70c2e015c968cb91cbd7f1b486933056d422337d75
Successfully built h2o


In [3]:
import sys
import h2o
from h2o.automl import H2OAutoML
import numpy as np
import pandas as pd

In [4]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.10" 2021-01-19; OpenJDK Runtime Environment (build 11.0.10+9-Ubuntu-0ubuntu1.18.04); OpenJDK 64-Bit Server VM (build 11.0.10+9-Ubuntu-0ubuntu1.18.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpyyexcz7s
  JVM stdout: /tmp/tmpyyexcz7s/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpyyexcz7s/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.0.4
H2O_cluster_version_age:,27 days
H2O_cluster_name:,H2O_from_python_unknownUser_a1cohq
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.180 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
train = h2o.import_file(path='/content/drive/MyDrive/Colab Notebooks/signate-414/train.csv')
test = h2o.import_file(path='/content/drive/MyDrive/Colab Notebooks/signate-414/test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
train.summary

index,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
200,9,125,74,0,0,28.5369,0.444902,45,1
3832,4,109,80,0,0,28.0477,0.238243,22,0
4927,4,88,78,39,0,52.3713,0.279471,26,0
4088,9,125,74,0,0,40.0627,0.203922,45,0
3644,5,107,78,44,284,52.9351,0.284959,45,1
3323,5,84,64,0,0,33.4595,0.505127,26,0
2203,2,138,86,30,0,32.8047,0.26927,27,1
565,1,123,52,0,0,39.865,0.714738,45,1
1386,1,138,76,30,0,39.6604,0.503107,23,0
4162,7,100,74,0,0,33.3964,0.238128,43,0


<bound method H2OFrame.summary of >

In [7]:
# Identify predictors an d response
x = train.drop('index').columns
y = "Outcome"
x.remove(y)

In [8]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()
#test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_models=30, seed=1)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [10]:
model_path = h2o.save_model(model=aml.leader, path='/content/drive/MyDrive/Colab Notebooks/signate-414/', force=True)
print(model_path)

/content/drive/MyDrive/Colab Notebooks/signate-414/StackedEnsemble_BestOfFamily_AutoML_20210228_152051


In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20210228_152051,0.821486,0.428396,0.594996,0.262316,0.370301,0.137122
GBM_grid__1_AutoML_20210228_152051_model_5,0.820907,0.424582,0.58933,0.250929,0.369912,0.136835
StackedEnsemble_AllModels_AutoML_20210228_152051,0.820648,0.429614,0.593548,0.25928,0.371004,0.137644
GBM_5_AutoML_20210228_152051,0.809689,0.430659,0.575849,0.27261,0.373437,0.139455
XGBoost_grid__1_AutoML_20210228_152051_model_7,0.809216,0.434282,0.581023,0.257528,0.374193,0.14002
GBM_grid__1_AutoML_20210228_152051_model_4,0.808685,0.431342,0.583842,0.272091,0.373116,0.139216
GBM_grid__1_AutoML_20210228_152051_model_2,0.808046,0.433917,0.580451,0.277226,0.373777,0.139709
XGBoost_grid__1_AutoML_20210228_152051_model_9,0.807399,0.436323,0.567389,0.2629,0.376512,0.141761
GBM_2_AutoML_20210228_152051,0.807365,0.433533,0.576414,0.270062,0.374247,0.140061
GBM_3_AutoML_20210228_152051,0.806534,0.434273,0.579296,0.268264,0.374398,0.140174


In [12]:
preds = aml.predict(test).as_data_frame()
preds.shape

stackedensemble prediction progress: |████████████████████████████████████| 100%


(2000, 3)

In [16]:
test_df = test.as_data_frame()
submit = pd.concat([test_df["index"], preds["predict"]], axis=1)
submit.to_csv('/content/drive/MyDrive/Colab Notebooks/signate-414/submit_automl_30-1.csv', header=False, index=False)

In [14]:
aml.leader.model_performance

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20210228_152051

No model summary for this model

ModelMetricsBinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.05086519890949009
RMSE: 0.2255331437050663
LogLoss: 0.21485741183735607
Null degrees of freedom: 2999
Residual degrees of freedom: 2994
Null deviance: 3299.5470284265884
Residual deviance: 1289.1444710241367
AIC: 1301.1444710241367
AUC: 0.98998173999686
AUCPR: 0.9722897253138467
Gini: 0.97996347999372

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.36260909051960727: 


,,0,1,Error,Rate
0,0,2211.0,72.0,0.0315,(72.0/2283.0)
1,1,52.0,665.0,0.0725,(52.0/717.0)
2,Total,2263.0,737.0,0.0413,(124.0/3000.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.362609,0.914718,208.0
1,max f2,0.274703,0.934806,238.0
2,max f0point5,0.517325,0.936805,160.0
3,max accuracy,0.362609,0.958667,208.0
4,max precision,0.952265,1.000000,0.0
5,max recall,0.114459,1.000000,335.0
6,max specificity,0.952265,1.000000,0.0
7,max absolute_mcc,0.362609,0.887594,208.0
8,max min_per_class_accuracy,0.301819,0.950504,225.0
9,max mean_per_class_accuracy,0.326307,0.951300,218.0



Gains/Lift Table: Avg response rate: 23.90 %, avg score: 26.76 %


...


Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.259336,0.586997,242.0
1,max f2,0.119985,0.710040,330.0
2,max f0point5,0.492668,0.584998,143.0
3,max accuracy,0.492668,0.807667,143.0
4,max precision,0.955475,1.000000,0.0
5,max recall,0.063738,1.000000,391.0
6,max specificity,0.955475,1.000000,0.0
7,max absolute_mcc,0.277887,0.444336,233.0
8,max min_per_class_accuracy,0.203412,0.740254,275.0
9,max mean_per_class_accuracy,0.150635,0.746251,306.0



Gains/Lift Table: Avg response rate: 23.90 %, avg score: 23.89 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.01,0.894307,3.626220,3.626220,0.866667,0.919294,0.866667,0.919294,0.036262,0.036262,262.622036,262.622036,0.034510
1,2,0.02,0.851069,3.068340,3.347280,0.733333,0.873679,0.800000,0.896486,0.030683,0.066946,206.834031,234.728033,0.061689
2,3,0.03,0.820202,3.207810,3.300790,0.766667,0.835932,0.788889,0.876302,0.032078,0.099024,220.781032,230.079033,0.090701
3,4,0.04,0.785824,2.649930,3.138075,0.633333,0.803591,0.750000,0.858124,0.026499,0.125523,164.993026,213.807531,0.112382
4,5,0.05,0.763190,2.928870,3.096234,0.700000,0.775622,0.740000,0.841624,0.029289,0.154812,192.887029,209.623431,0.137729
5,6,0.10,0.599185,2.789400,2.942817,0.666667,0.684171,0.703333,0.762897,0.139470,0.294282,178.940028,194.281729,0.255298
6,7,0.15,0.481891,2.231520,2.705718,0.533333,0.540973,0.646667,0.688922,0.111576,0.405858,123.152022,170.571827,0.336213
7,8,0.20,0.385753,1.813110,2.482566,0.433333,0.433188,0.593333,0.624989,0.090656,0.496513,81.311018,148.256625,0.389636
8,9,0.30,0.257457,1.645746,2.203626,0.393333,0.317611,0.526667,0.522530,0.164575,0.661088,64.574616,120.362622,0.474491
9,10,0.40,0.187404,1.087866,1.924686,0.260000,0.221183,0.460000,0.447193,0.108787,0.769874,8.786611,92.468619,0.486037


<bound method ModelBase.model_performance of >